In [2]:
from tqdm import tqdm_notebook

In [59]:
from queue import Queue


already_visited = []
valve_maps = {}
pressure_map = {}

path_maps = {}

with open('input.txt') as file:
    for f in file:
        current_valve = f.split()[1]
        flow_rate = f.split()[4][5:-1]
        other_valves = [i.strip(',') for i in f.split()[9:]]
        all_valves = [current_valve] + other_valves

        valve_maps[current_valve] = valve_maps.get(current_valve, []) + other_valves
        for valve in other_valves:
            valve_maps[valve] = valve_maps.get(valve, []) + [current_valve]
        pressure_map[current_valve] = int(flow_rate)
        
for key, valve in valve_maps.items():
    valve_maps[key] = set(valve)
    
def leastDistance(graph, source):
    Q = Queue()
    # create a dictionary with large distance(infinity) of each vertex from source
    distance = {k: 9999999 for k in graph.keys()}
    visited_vertices = set()
    Q.put(source)
    visited_vertices.update({0})
    while not Q.empty():
        vertex = Q.get()
        if vertex == source:
            distance[vertex] = 0
        for u in graph[vertex]:
            if u not in visited_vertices:
                # update the distance
                if distance[u] > distance[vertex] + 1:
                    distance[u] = distance[vertex] + 1
                Q.put(u)
                visited_vertices.update({u})
    return distance
    
def calculate_pressure_path(path, valves_map, pressure_map, current_time=30):
    current_node = 'AA'
    total_pressure = 0
    for i in path:
        map_from_here = leastDistance(valves_map, current_node)
        current_time -= map_from_here[i] + 1
        total_pressure += (current_time) * pressure_map[i]
        current_node = i
    return total_pressure, current_time

# At any point, choose the top 3 choices, as defined by expected remaining value
starting_node = 'AA'
paths_so_far = []
finalized_paths = []
all_nodes = list(valve_maps.keys())
pressure_nodes = [i for i in all_nodes if pressure_map[i] > 0]

# while True:
#     if len(paths_so_far) == 0:
#         if len(finalized_paths) == 0:
#             candidate_destinations = []
#             for destination in [i for i in all_nodes if i != starting_node]:
#                 new_path = [destination]
#                 pressure, time = calculate_pressure_path(new_path, valve_maps, pressure_map)
#                 candidate_destinations.append((destination, pressure))
#             best_candidates = sorted(candidate_destinations, key = lambda x: x[1])[-100:]
#             for candidate, pressure in best_candidates:
#                 paths_so_far.append(([candidate], pressure))
#         else:
#             break
#     else:
#         new_paths_so_far = []
#         best_paths_so_far = sorted(paths_so_far, key = lambda x: x[1])[-100:]
#         for path, pressure in best_paths_so_far:
#             current_node = path[-1]
#             other_paths_possible = False
#             candidate_destinations = []
#             for destination in [i for i in all_nodes if i not in path]:
#                 new_path = path + [destination]
#                 pressure, time = calculate_pressure_path(new_path, valve_maps, pressure_map)
#                 if time >= 0:
#                     print(f'Adding new path {new_path}')
#                     new_paths_so_far.append((new_path, pressure))
#                     other_paths_possible = True                    
#             if not other_paths_possible:
#                 finalized_paths.append(path)
#         paths_so_far = new_paths_so_far

# highest_pressure = 0
# for path in finalized_paths:
#     pressure, time = calculate_pressure_path(path, valve_maps, pressure_map)
#     if pressure > highest_pressure:
#         highest_pressure = pressure
#         print(f'Exceeded by path {path}')
        
# print(highest_pressure)

paths_so_far = []

while True:
    if len(paths_so_far) == 0:
        if len(finalized_paths) == 0:
            candidate_destinations = []
            for destination in [i for i in pressure_nodes if i != starting_node]:
                new_path = [destination]
                for ele_destination in [i for i in pressure_nodes if i not in [starting_node, destination]]:
                    ele_path = [ele_destination]
                    me_pressure, me_time = calculate_pressure_path(new_path, valve_maps, pressure_map, current_time=26)
                    ele_pressure, ele_time = calculate_pressure_path(ele_path, valve_maps, pressure_map, current_time=26)
                    total_pressure = me_pressure + ele_pressure
                    paths_so_far.append((new_path, ele_path, total_pressure))
#             print(paths_so_far)
        else:
            break
    else:
        print('Entering this loop')
        new_paths_so_far = []
        best_paths_so_far = sorted(paths_so_far, key = lambda x: x[2])
#         print(best_paths_so_far)
        
        other_me_paths_possible = False
        other_ele_paths_possible = False
        prev_me_already_added = False
        prev_ele_already_added = False
        candidate_me_paths = []
        candidate_ele_paths = []
        candidate_solutions = []
        
        for me_path, ele_path, pressure in tqdm_notebook(best_paths_so_far):
            other_paths_possible = False
            for possible_destinations in [i for i in pressure_nodes if i not in me_path + ele_path]:
                candidate_me_path = me_path + [possible_destinations]
                candidate_ele_path = ele_path + [possible_destinations]
                me_pressure, me_time = calculate_pressure_path(candidate_me_path, valve_maps, pressure_map, current_time=26)
                ele_pressure, ele_time = calculate_pressure_path(candidate_ele_path, valve_maps, pressure_map, current_time=26)
                
                prev_me_pressure, prev_me_time = calculate_pressure_path(me_path, valve_maps, pressure_map, current_time=26)
                prev_ele_pressure, prev_ele_time = calculate_pressure_path(ele_path, valve_maps, pressure_map, current_time=26)
                
                if me_time >= 0:
                    candidate_me_paths.append((candidate_me_path, ele_path, me_pressure + prev_ele_pressure))
                    other_paths_possible = True
                
                if ele_time >= 0:
                    candidate_ele_paths.append((me_path, candidate_ele_path, prev_me_pressure + ele_pressure))
                    other_paths_possible = True
            if not other_paths_possible:
                
                finalized_paths.append((me_path, ele_path, pressure))
        
        paths_so_far = sorted(candidate_me_paths, key = lambda x: x[2])[-2000:] + sorted(candidate_ele_paths, key = lambda x: x[2])[-2000:]
        
#         if other_paths_possible:
#             print('Other paths are possible')
#             # Get the best me paths and ele paths
#             best_me_paths = sorted(candidate_me_paths, key=lambda x: x[1])[-10:]
#             best_ele_paths = sorted(candidate_ele_paths, key=lambda x: x[1])[-10:]
            
#             for me_path in best_me_paths:
#                 for ele_path in best_ele_paths:
#                     if set(me_path[0] + ele_path[0]) == me_path[0] + ele_path[0]:
#                         new_paths_so_far.append((me_path[0], ele_path[0], me_path[1] + ele_path[1]))
                        
#             paths_so_far = new_paths_so_far
            
#         else:
#             paths_so_far = []
#             finalized_paths.append((me_path, ele_path, pressure))
            
                
#             for me_destination in [i for i in all_nodes if i not in me_path + ele_path]:
#                 for ele_destination in [i for i in all_nodes if i not in me_path + ele_path]:
#                     if me_destination == ele_destination:
#                         continue
#                     candidate_me_path = me_path + [me_destination]
#                     candidate_ele_path = ele_path + [ele_destination]
#                     me_pressure, me_time = calculate_pressure_path(candidate_me_path, valve_maps, pressure_map, current_time=26)
#                     ele_pressure, ele_time = calculate_pressure_path(candidate_ele_path, valve_maps, pressure_map, current_time=26)
#                     if me_time >= 0:
#                         new_me_path = candidate_me_path
#                         other_paths_possible = True
#                     else:
#                         new_me_path = me_path
                    
#                     if ele_time >= 0:
#                         new_ele_path = candidate_ele_path
#                         other_paths_possible = True
#                     else:
#                         new_ele_path = ele_path
                    
#                     if (me_time >= 0) or (ele_time >= 0):
#                         new_me_pressure, new_me_time = calculate_pressure_path(new_me_path, valve_maps, pressure_map, current_time=26)
#                         new_ele_pressure, new_ele_time = calculate_pressure_path(new_ele_path, valve_maps, pressure_map, current_time=26)
#                         new_pressure = new_me_pressure + new_ele_pressure
#                         new_paths_so_far.append((new_me_path, new_ele_path, new_pressure))
# #                         print(f'Adding new path {new_me_path, new_ele_path}')
#             if not other_paths_possible:
#                 finalized_paths.append((me_path, ele_path, pressure))
#         paths_so_far = new_paths_so_far

highest_pressure = sorted(finalized_paths, key = lambda x: x[2])[-1]
        
print(highest_pressure)

Entering this loop


/var/folders/d5/f5qxbxd93vg0sqk_wkcp0g0h0000gn/T/ipykernel_49640/3610250.py:133: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for me_path, ele_path, pressure in tqdm_notebook(best_paths_so_far):


  0%|          | 0/210 [00:00<?, ?it/s]

Entering this loop


  0%|          | 0/4000 [00:00<?, ?it/s]

...

Entering this loop


  0%|          | 0/1449 [00:00<?, ?it/s]

Entering this loop


  0%|          | 0/264 [00:00<?, ?it/s]

(['YI', 'UA', 'AW', 'EL', 'OY', 'FL', 'EG'], ['OZ', 'FX', 'VA', 'GQ', 'TU', 'WW'], 2591)


In [34]:
calculate_pressure_path(['YI', 'UA', 'AW', 'EL', 'OY', 'FL'], valve_maps, pressure_map, current_time=26)

(1473, 6)

In [35]:
calculate_pressure_path(['EG', 'XY', 'CR', 'OZ', 'FX'], valve_maps, pressure_map, current_time=26)

(954, 2)

In [ ]:
# Find groups of 3 that work very well together


In [55]:
calculate_pressure_path(['YI', 'UA', 'AW', 'EL', 'OY', 'FL', 'EG'], valve_maps, pressure_map, current_time=26)

(1515, 2)

In [58]:
{key: value for key,value in pressure_map.items() if key in pressure_nodes}

{'EG': 21,
 'EL': 16,
 'FL': 22,
 'VA': 20,
 'UA': 19,
 'WW': 4,
 'XY': 24,
 'FX': 14,
 'OY': 13,
 'GQ': 18,
 'AW': 25,
 'CR': 8,
 'TU': 7,
 'OZ': 10,
 'YI': 15}

In [56]:
{key: value for key,value in leastDistance(valve_maps, 'AA').items() if key in pressure_nodes}

{'EG': 5,
 'CR': 2,
 'FL': 8,
 'OY': 6,
 'EL': 8,
 'AW': 8,
 'YI': 4,
 'FX': 4,
 'UA': 6,
 'OZ': 2,
 'WW': 2,
 'GQ': 7,
 'TU': 3,
 'VA': 6,
 'XY': 8}